In [1]:
using CSV, DataFrames, Random, ScikitLearn, Plots, CategoricalArrays

In [2]:
data = CSV.read("data/df_new.csv",DataFrame);

In [3]:
names(data)

16-element Vector{String}:
 "PatientId"
 "AppointmentID"
 "Gender"
 "ScheduledDay"
 "AppointmentDay"
 "Age"
 "Neighbourhood"
 "Scholarship"
 "Hipertension"
 "Diabetes"
 "Alcoholism"
 "Handcap"
 "SMS_received"
 "NoShow"
 "AwaitingTime"
 "NbPrevious"

In [4]:
first(data,5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay
,Int64,Int64,String1,String31,String31
1,30845,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 23:59:59+00:00
2,55325,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 23:59:59+00:00
3,15722,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 23:59:59+00:00
4,9018,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 23:59:59+00:00
5,23941,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 23:59:59+00:00


In [5]:
X = select(data, Not([:AppointmentID, :ScheduledDay, :AppointmentDay, :NoShow]))
y = data[:,:NoShow];

In [6]:
Random.seed!(1);
(train_X, train_y), (test_X, test_y) = IAI.split_data(:classification, X, y,  train_proportion=0.6, seed=1);

In [7]:
function eval(model, prt)
    acc_train = IAI.score(model, train_X, train_y, criterion=:misclassification)
    auc_train = IAI.score(model, train_X, train_y, criterion=:auc)
    acc_test = IAI.score(model, test_X, test_y, criterion=:misclassification)
    auc_test = IAI.score(model, test_X, test_y, criterion=:auc)
    if prt
        println("Training")
        println("Accuracy : $acc_train")
        println("AUC : $auc_train \n")
        println("Testing")
        println("Accuracy : $acc_test")
        println("AUC : $auc_test")
    end
    return (acc_train, auc_train, acc_test, auc_test)
end

eval (generic function with 2 methods)

In [8]:
OCT_miscla = IAI.GridSearch(
    IAI.OptimalTreeClassifier(max_depth=3, minbucket=5, criterion=:auc, max_categoric_levels_before_warning = 100))
IAI.fit!(OCT_miscla,train_X,train_y, validation_criterion=:auc)
OCT_miscla_opti = IAI.get_learner(OCT_miscla)

eval(OCT_miscla_opti, true);

LoadError: ArgumentError: criterion must be in (:misclassification, :gini, :entropy) for OptimalTrees.OptimalTreeClassifier

In [9]:
OCT_miscla_opti

LoadError: UndefVarError: OCT_miscla_opti not defined

In [ ]:
gr = IAI.GridSearch(
    IAI.OptimalTreeClassifier(criterion=:misclassification, max_categoric_levels_before_warning = 20),
    max_depth = 1:10,
    minbucket = 1:10,
)
IAI.fit!(gr, train_X, train_y, validation_criterion=:misclassification)
lnr = IAI.get_learner(gr);

┌ Warning: The following categoric features have more than 20 levels:
│ - Neighbourhood
│ 
│ We recommend extreme caution when using categoric features with many levels inside Optimal Trees, for more information and advice on how to handle such features, please refer to this link:
│ 
│ https://docs.interpretable.ai/dev/OptimalTrees/tips/#Categorical-Variables-with-Many-Levels
│ 
│ You can suppress this warning by increasing the value of the parameter `max_categoric_levels_before_warning` on the Optimal Tree learner.
└ @ nothing nothing:nothing
┌ Warning: The following categoric features have more than 20 levels:
│ - Neighbourhood
│ 
│ We recommend extreme caution when using categoric features with many levels inside Optimal Trees, for more information and advice on how to handle such features, please refer to this link:
│ 
│ https://docs.interpretable.ai/dev/OptimalTrees/tips/#Categorical-Variables-with-Many-Levels
│ 
│ You can suppress this warning by increasing the value of the param

In [ ]:
(_,_, acc, auc) = eval(lnr, true);

In [ ]:
lnr